In [10]:
# Import the necessary Libraries
import torch
import pandas as pd
from torchvision import transforms, utils
from torch.autograd import Variable
import torch.utils.data as data_utils
import numpy as np
import matplotlib.pyplot as graph
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score , classification , classification_report
import warnings
warnings.filterwarnings("ignore",category=FutureWarning)

In [11]:
# Load the dataset
iris_df = pd.read_csv('iris.csv')


In [12]:
iris_df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [13]:
# Convert the categorical label to Numerical
# Define the Label Encoder
le = preprocessing.LabelEncoder()
labels = list(iris_df["Species"].unique())
le.fit(labels)
iris_df["Species"] = le.transform(iris_df["Species"])
iris_df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,0
1,2,4.9,3.0,1.4,0.2,0
2,3,4.7,3.2,1.3,0.2,0
3,4,4.6,3.1,1.5,0.2,0
4,5,5.0,3.6,1.4,0.2,0


In [14]:
iris_df.columns

Index(['Id', 'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm',
       'Species'],
      dtype='object')

In [15]:
# Seperate the features and labels seperate
iris_df.columns

iris_features = iris_df[['SepalWidthCm','SepalWidthCm','PetalWidthCm','PetalWidthCm']]
iris_labels = iris_df['Species']



In [16]:
# Split into train and test
train_feat , test_feat, train_label, test_label = train_test_split(iris_features, iris_labels, test_size=0.20, random_state=42)
logreg = LogisticRegression()      # Object-Creation
logreg.fit(train_feat,train_label)        # Fitting the data into the object
predict = logreg.predict(test_feat)  

logreg_train_score = round(logreg.score(train_feat,train_label)*100,2)
logreg_test_score = round(logreg.score(test_feat,test_label)*100,2)
accuracy = accuracy_score(test_label,predict)
class_report = classification_report(test_label,predict)

print('train_score : ',logreg_train_score)
print('test score :' , logreg_test_score)
print('Accuracy : ' , accuracy)

train_score :  95.0
test score : 96.67
Accuracy :  0.9666666666666667


In [17]:
# Convert the train into tensors and load as Dataloaders

train_feat = torch.Tensor(train_feat.values)
train_label = torch.LongTensor(train_label.values)
train_data = data_utils.TensorDataset(train_feat,train_label)
train_loader = data_utils.DataLoader(dataset = train_data,batch_size=64,shuffle=True)

In [18]:
# Convert the test into tensors and load as Dataloaders

test_feat = torch.Tensor(test_feat.values)
test_label = torch.LongTensor(test_label.values)
test_data = data_utils.TensorDataset(test_feat,test_label)
test_loader = data_utils.DataLoader(dataset = test_data,batch_size=64,shuffle=True)


In [19]:
# Define the neural Network

class IrisNet(torch.nn.Module):
    def __init__(self,input_layer,hidden_1_size,hidden_2_size,output_layer):
        super(IrisNet,self).__init__()
        self.first_layer = torch.nn.Linear(input_layer,hidden_1_size)
        self.first_layer_activation = torch.nn.ReLU()
        self.second_layer = torch.nn.Linear(hidden_1_size,hidden_2_size)
        self.second_layer_activation = torch.nn.ReLU()
        self.third_layer = torch.nn.Linear(hidden_2_size,output_layer)
        self.third_layer_activation = torch.nn.Sigmoid()
        
    def forward(self,x):
        output_1 = self.first_layer(x)
        output_1 = self.first_layer_activation(output_1)
        output_2 = self.second_layer(output_1)
        output_2 = self.output_layer_activation(output_2)
        output_3 = self.third_layer(output_2)
        output_3 = self.third_layer_activation(output_3)
        return output_3


In [20]:
# Declare the model and its hyperparameters
#Specifying the no of neurons in every layer
model = IrisNet(4,100,40,3)

#Defining the loss function for the model
crieterion = torch.nn.CrossEntropyLoss()   

#defining the learning rate
learning_rate = 0.001

#defining the optimizer 
optimiser = torch.optim.SGD(model.parameters(),lr = learning_rate)


In [21]:
# Some utility variables
no_of_epoch = 20

training_loss=[]
training_accuracy=[]
testing_loss=[]
testing_accuracy=[]
epoch_list=[]

In [22]:
# training script
# Graph Based Execution

for epochs in range(no_of_epoch):
    totally_trained = 0
    currently_trained= 0
    
    for i , (train_features,train_labels) in enumerate(train_loader):
        #setting up input
        train_features = Variable(train_features)
        train_labels = Variable(train_labels)
        train_labels = train_labels.reshape(-1)
        
        
        #setting model for training
        model.train()
        
        #Initialise gradient to zero
        optimiser.zero_grad()
        
        #Predicting output
        output = model(train_features)
        
        # calculate loss
        loss = crieterion(output,train_labels)
        
        #backward function
        loss.backward()
        
        #optimisation
        optimiser.step()
        
        #getting Prediction labels
        total_trained += train_labels.size(0)



AttributeError: 'IrisNet' object has no attribute 'output_layer_activation'